In [24]:
from matplotlib import pyplot as plt
import psycopg2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432", 
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Création d'un curseur pour exécuter des requêtes SQL
cur = conn.cursor()

# Requête SQL pour sélectionner les données de la table securite
query_securite = "SELECT code_postal, tauxpourcent FROM securite;"
cur.execute(query_securite)
rows_securite = cur.fetchall()

# Requête SQL pour sélectionner les données du premier tour (ou du second tour si nécessaire)
query_election = "SELECT code_postal, \"% Voix/Ins 1\", \"% Voix/Ins 2\" FROM election_2022_t2"
cur.execute(query_election)
rows_election = cur.fetchall()

# Fermeture du curseur et de la connexion à la base de données
cur.close()
conn.close()

# Création des DataFrames à partir des résultats des requêtes
df_securite = pd.DataFrame(rows_securite, columns=['code_postal', 'tauxpourcent'])
df_election = pd.DataFrame(rows_election, columns=['code_postal', '% Voix/Ins 1', '% Voix/Ins 2'])

# Fusionner les données de sécurité avec les données de l'élection sur la colonne code_postal
df_merged = pd.merge(df_securite, df_election, on='code_postal')

# Sélectionner uniquement les données pour La République En Marche
# (dans ce cas, nous considérons que toutes les données de df_merged appartiennent au parti La République En Marche)
df_party = df_merged

# Séparer les features (X) et la target (y)
X = df_party[['tauxpourcent']].values
y = df_party['% Voix/Ins 1'].values

# Diviser les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle de régression linéaire sur l'ensemble d'entraînement
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test pour évaluer la performance du modèle
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Erreur quadratique moyenne sur l'ensemble de test:", mse)

# Utiliser le modèle entraîné pour prédire le vote de La République En Marche dans le département 89
dept_89_securite = df_securite[df_securite['code_postal'] == 89]['tauxpourcent'].values.reshape(-1, 1)
prediction_dept_89 = model.predict(dept_89_securite)
print("Prédiction du vote de La République En Marche dans le département 89:", prediction_dept_89[0])


Erreur quadratique moyenne sur l'ensemble de test: 95.53324376617026
Prédiction du vote de La République En Marche dans le département 89: 37.21104222354786
